In [1]:
#已知問題：合計股數用加總，當週有違約交割時總數會有誤差

import requests
import pandas as pd
import dateutil
from bs4 import BeautifulSoup as BS
from pymongo import MongoClient

def read_mongo(db, collection, query, host, port, username, password, no_id):
    """ Read from Mongo and Store into DataFrame """

    # Connect to mongo
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df
    
def dataframe_to_mongo(select_df, db, collection, host, port, username, password):

    # connect to mongo
    database = _connect_mongo(host=host, port=port, username=username, password=password, db=db)
    collection = database[collection]   
    
    records = select_df.to_dict('records')  # 參數 record 代表把列轉成個別物件
    collection.insert_many(records)
    
def _connect_mongo(host, port, username, password, db):
    # Connect to mongo 

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)

    return conn[db]

def read_tickers():
    df = pd.read_csv('stocklist.csv')
    tickerlist = df['股票代號']
    
    return tickerlist

host = 'localhost'
port = 27017
query = {}
username = None
password = None
no_id = True    
db = 'twstock'
collection = 'chips'

fetch_month = ['20171215']
tickers = read_tickers()
#tickers = [1215,1216] #for test

url = 'https://www.tdcc.com.tw/smWeb/QryStock.jsp?'

for ticker in tickers:    
    for date in fetch_month:
        get = 'SCA_DATE={}&SqlMethod=StockNo&StockNo={}&StockName=&sub=%ACd%B8%DF'.format(date, ticker)
        html = url + get
       
        while True:
            try:
                df = pd.read_html(html)[0]
                #print('{} {} ok!'.format(date, ticker))
                break
            except: 
                print("{} {} wrong!".format(date, ticker))
            
        #資料梳理
        final_df = df.loc[10:24]
        final_df = final_df.reset_index()
        final_df = final_df.T
        final_df = final_df[1:6]
        number = final_df.T[2]
        share = final_df.T[3]
        ratio = final_df.T[4]
            
        number = pd.to_numeric(number)
        number.index = ['number_1','number_2','number_3','number_4','number_5','number_6','number_7','number_8','number_9','number_10','number_11','number_12','number_13','number_14','number_15']
        number['number_16']= sum(number)
        share = pd.to_numeric(share)
        share.index = ['shares_1','shares_2','shares_3','shares_4','shares_5','shares_6','shares_7','shares_8','shares_9','shares_10','shares_11','shares_12','shares_13','shares_14','shares_15']
        share['shares_16']= sum(share)
        ratio = pd.to_numeric(ratio)
        ratio.index = ['ratio_1','ratio_2','ratio_3','ratio_4','ratio_5','ratio_6','ratio_7','ratio_8','ratio_9','ratio_10','ratio_11','ratio_12','ratio_13','ratio_14','ratio_15']
            
        ratio = ((ratio.to_frame()).T).reset_index()
        number = ((number.to_frame()).T).reset_index()
        share = ((share.to_frame()).T).reset_index()
            
        ratio.pop('index')
        number.pop('index')
        share.pop('index')
            
        final_df = pd.concat([number, ratio, share], axis = 1)
        final_df['ticker'] = int(ticker)
        final_df['date'] = date
        final_df['date'] = pd.to_datetime(final_df['date'])
            
        #存到mongodb
        dataframe_to_mongo(final_df, db, collection, host, port, username, password)
        
print('done!')

20171215 1101 wrong!
20171215 1459 wrong!
20171215 8928 wrong!
